# 下载数据集，并对数据集进行处理！

### 1. 下载数据集

In [15]:
from datasets import load_dataset

dataset_path = "/raid/gfc/llm/datasets/Chinese-medical-dialogue"
ds = load_dataset("ticoAg/Chinese-medical-dialogue", cache_dir=dataset_path)

# 查看数据集的基本信息
print("数据集结构：")
for k, v in ds.items():
    print(f"{k}: {v}")

# 获取数据集的大小
print(f"\n训练集大小: {len(ds['train'])}")

# 查看数据集的列名（特征）
print("\n数据集的特征：")
print(ds['train'].features)

数据集结构：
train: Dataset({
    features: ['instruction', 'input', 'output', 'history'],
    num_rows: 799743
})

训练集大小: 799743

数据集的特征：
{'instruction': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None), 'history': Value(dtype='null', id=None)}


### 2. 清洗数据集

##### 2.1 缺失值处理

In [16]:
# 数据清洗
def is_valid(example):
    # 三项都不能为空且不是纯空格
    return all([
        example['instruction'] and example['instruction'].strip(),
        example['input'] and example['input'].strip(),
        example['output'] and example['output'].strip()
    ])

dataset = ds['train']
print(f"原始样本数: {len(dataset)}")

# 过滤空值
ds_clean = dataset.filter(is_valid)
print(f"清洗后样本数: {len(ds_clean)}")

print(ds_clean[0])
print(ds_clean[0]['instruction'])
print(type(ds_clean))

原始样本数: 799743
清洗后样本数: 799736
{'instruction': '小儿肥胖超重该如何治疗', 'input': '女宝宝，刚7岁，这一年，察觉到，我家孩子身上肉很多，而且，食量非常的大，平时都不喜欢吃去玩，请问：小儿肥胖超重该如何治疗。', 'output': '孩子出现肥胖症的情况。家长要通过孩子运功和健康的饮食来缓解他的症状，可以先让他做一些有氧运动，比如慢跑，爬坡，游泳等，并且饮食上孩子多吃黄瓜，胡萝卜，菠菜等，禁止孩子吃一些油炸食品和干果类食物，这些都是干热量高脂肪的食物，而且不要让孩子总是吃完就躺在床上不动，家长在治疗小儿肥胖期间如果孩子情况严重就要及时去医院在医生的指导下给孩子治疗。', 'history': None}
小儿肥胖超重该如何治疗
<class 'datasets.arrow_dataset.Dataset'>


##### 2.2 格式化规范
去除多余空格、特殊符号。
统一全角/半角、简繁体（如有需要）。
统一标点符号。

In [17]:
import re
import jaconv
from zhconv import convert

def normalize_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9，。！？、；：“”‘’（）《》【】]', '', text)
    text = jaconv.z2h(text, kana=False, ascii=True, digit=True)
    text = convert(text, 'zh-cn')
    return text

def normalize_example(example):
    for col in ['instruction', 'input', 'output']:
        example[col] = normalize_text(str(example[col]))
    return example

# 推荐：直接在 HuggingFace Dataset 上并行处理
ds_clean = ds_clean.map(normalize_example, num_proc=4)  # 可根据CPU核数调整num_proc

print("数据格式化完成！")

Map (num_proc=4): 100%|██████████| 799736/799736 [03:01<00:00, 4398.32 examples/s]

数据格式化完成！


##### 2.3 去除冗余数据集

In [19]:
# 去重（以instruction+input+output为唯一标识）
import pandas as pd
from simhash import Simhash

df = ds_clean.to_pandas()
df = df.drop_duplicates(subset=['instruction', 'input', 'output'])
print(f"去重后样本数: {len(df)}")

# 近似去重示例（SimHash）
def get_simhash(text):
    return Simhash(text).value

df['simhash'] = df.apply(lambda row: get_simhash(row['instruction'] + row['input'] + row['output']), axis=1)
df = df.drop_duplicates(subset=['simhash'])
print(f"近似去重后样本数: {len(df)}")

去重后样本数: 752436
近似去重后样本数: 750026


In [20]:
import os

# 存储格式化以后的数据集
formated_dataset_path = os.path.join(dataset_path, "formatted_dataset(quchong+simhash).csv")

ds_clean.to_csv(formated_dataset_path, index=False, encoding='utf-8')
print(f"格式化后的数据集已保存到: {formated_dataset_path}")

Creating CSV from Arrow format: 100%|██████████| 800/800 [00:16<00:00, 48.12ba/s]

格式化后的数据集已保存到: /raid/gfc/llm/datasets/Chinese-medical-dialogue/formatted_dataset(quchong+simhash).csv


In [ ]:
# 可选：过滤过短/过长
df = df[df['input'].str.len() < 5]
df = df[df['output'].str.len() < 5]

# 保存清洗后的数据
df.to_csv("/raid/gfc/llm/datasets/Chinese-medical-dialogue/cleaned_train.csv", index=False, encoding='utf-8')

### 3. 处理数据集